In [ ]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
!pip install torchdiffeq
from torchdiffeq import odeint#_adjoint as odeint
import numpy as np
from scipy import io
import argparse
import os
import pickle
import matplotlib.pyplot as plt
from timeit import default_timer
from google.colab import drive
drive.mount('/content/drive')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
directory = "drive/MyDrive/"#"spring-runs_train/" #/mnt/data/datasets/physical_system_benchmark/experiment_output/spring/spring/run/data_gen/spring-runs_train-spring-n1000-t805-n0_00001/"
npz = np.load(directory + "trajectories.npz")
nb_trajs = 1000
start_traj = 0
p1, q1, dqdt1, dpdt1, t1 = np.zeros((nb_trajs,805)), np.zeros((nb_trajs,805)), np.zeros((nb_trajs,805)), np.zeros((nb_trajs,805)), np.zeros((nb_trajs,805))
with np.load(directory + "trajectories.npz") as npz:
  for i in range(start_traj, start_traj + nb_trajs):
    p1[i:i+1,:], q1[i:i+1,:], dqdt1[i:i+1,:], dpdt1[i:i+1,:], t1[i:i+1,:] = npz[npz.files[5*i+0]].T, npz[npz.files[5*i+1]].T, npz[npz.files[5*i+2]].T, npz[npz.files[5*i+3]].T, npz[npz.files[5*i+4]].T #The first trajectory
q_exact = torch.tensor(q1, dtype = torch.float32)
p_exact = torch.tensor(p1, dtype = torch.float32)
t = torch.tensor(t1[0], dtype = torch.float32)
exact = torch.cat((q_exact.reshape(nb_trajs,-1,1), p_exact.reshape(nb_trajs,-1,1)), dim=-1)

In [ ]:
test_trajs = 100
train_loader = DataLoader(exact[:(nb_trajs-test_trajs)], batch_size=64)
test_loader  = DataLoader(exact[(nb_trajs-test_trajs):], batch_size=64)

In [ ]:
print(exact.shape)
print(len(exact[(nb_trajs-test_trajs):]))

In [ ]:
class Lyapunov(nn.Module):
  def __init__(self, in_nodes, layers, activation):
    super(Lyapunov, self).__init__()
    self.in_nodes = in_nodes
    self.layers = layers
    self.act = activation
    self.w0 = nn.Linear(in_nodes, in_nodes)
    self.w1 = nn.Linear(in_nodes, 1)
    self.u1 = nn.Parameter(torch.Tensor(1, in_nodes))
    nn.init.xavier_uniform_(self.u1)
    self.g0 = self.g(torch.zeros(in_nodes))
  def g(self, x):
    z = self.act(self.w0(x))
    z = self.act(self.w1(x)  + F.linear(z, self.u1.exp()))
    return z
  def forward(self, x):
    return self.act(self.g(x) - self.g0) + torch.sum(x**2, dim=1).reshape(-1,1)/100
# class Lyapunov(nn.Module):
#   def __init__(self, in_nodes, layers, activation):
#     super(Lyapunov, self).__init__()
#     self.in_nodes = in_nodes
#     self.layers = layers
#     self.act = activation
#     self.W = [nn.Linear(in_nodes, in_nodes)]
#     self.U = []
#     for i in range(layers):
#       self.W.append(nn.Linear(in_nodes, in_nodes))
#       self.U.append(nn.Linear(in_nodes, in_nodes))
#     self.W.append(nn.Linear(in_nodes, 1))
#     self.U.append(nn.Linear(in_nodes, 1))
#     self.g0 = self.g(torch.zeros(in_nodes))
#   def g(self, x):
#     z = self.act(self.W[0](x))
#     for i in range(self.layers):
#       z = self.act(self.W[i+1](x) + self.U[i](z)**2)
#     z = self.act(self.W[self.layers+1](x) + self.U[self.layers](z)**2)
#     return z
#   def forward(self, x):
#     return self.act(self.g(x) - self.g0) + torch.norm(x, p=2)/100

In [ ]:
class ODEFunc(nn.Module):
    def __init__(self, in_nodes, hid_nodes, v_layers, v_act):
        super(ODEFunc, self).__init__()
        self.fc0 = nn.Linear(in_nodes+1, hid_nodes)
        self.fc1 = nn.Linear(hid_nodes, in_nodes)
        self.V = Lyapunov(in_nodes, v_layers, v_act)
    def forward(self, t, x):
        x = x.requires_grad_(True)
        tt = torch.ones_like(x[...,:1], requires_grad=True)*t
        xt = torch.cat((tt, x), dim=-1)
        fx = self.fc0(xt)
        fx = F.gelu(fx)
        fx = self.fc1(fx)
        Vx = self.V(x)
        grad_Vx = torch.autograd.grad(Vx, x, torch.ones(Vx.shape), create_graph=True, allow_unused=True, retain_graph=True)[0]
        alpha = 1
        fx = fx - grad_Vx*torch.relu(torch.sum(grad_Vx*fx,dim=1).reshape(-1,1) + alpha*Vx)/torch.sum(grad_Vx**2, dim=1).reshape(-1,1)
        return fx

In [ ]:
class NODE(nn.Module):
    def __init__(self, in_nodes, hid_nodes, tol, v_layers, v_act):
        super(NODE, self).__init__()
        self.ode = ODEFunc(in_nodes, hid_nodes, v_layers, v_act)
        self.fc0 = nn.Linear(2, in_nodes)
        self.fc1 = nn.Linear(in_nodes, 2)
        self.tol = tol
    def forward(self, x):
        x = x.requires_grad_(True)
        x = x.reshape(x.shape[0], -1)
        x = self.fc0(x)
        x0 = x
        x = odeint(self.ode, x, torch.tensor([0., 1.], device=x.device), atol=self.tol, rtol=self.tol)[-1]
        x = self.fc1(x)
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# model = NODE(32, 128, 1e-5, 3, nn.Softplus()).to(device)
latent_channels = 32
hidden_layers = 128
odefunc = ODEFunc(latent_channels, hidden_layers, 2, nn.ReLU())
prelayer = nn.Linear(2, latent_channels)
postlayer = nn.Linear(latent_channels, 2)
params = list(odefunc.parameters()) + list(prelayer.parameters()) + list(postlayer.parameters())
optim = torch.optim.Adam(params, lr=1e-3)
loss_fn = nn.MSELoss()
epochs = 50
tol = 1e-5
batch_train_losses = []
train_losses = []
test_losses = []
accuracies = []

In [ ]:
Nt = 805//35
alpha = 100
for i in range(epochs):
    t = default_timer()
    train_loss = 0
    test_loss = 0
    for x in train_loader:
        x0 = x[:,0,:].to(device).requires_grad_(True)
        z0 = prelayer(x0)
        zt = odeint(odefunc, z0, torch.linspace(0,1,Nt), atol=tol, rtol=tol)
        pred = postlayer(zt)
        pred = pred.permute(1,0,2)
        loss = loss_fn(pred,x[:,::35,:])
        # loss = torch.tensor(0.0, requires_grad=True, device=device)
        # loss_now = loss_fn(pred[0], x[0])
        # for i in range(1,pred.shape[0]):
        #   loss_prev = loss_now
        #   loss_now = loss_fn(pred[i], x[i])
        #   loss = loss + torch.relu(loss_now + (alpha-1)*loss_prev)
        # optim.zero_grad()
        loss.backward()
        optim.step()
        train_loss += loss.item()
        batch_train_losses.append(loss.item())
    for x in test_loader:
        x0 = x[:,0,:].to(device).requires_grad_(True)
        z0 = prelayer(x0)
        zT = odeint(odefunc, z0, torch.linspace(0,1,Nt), atol=tol, rtol=tol)
        pred = postlayer(zT)
        pred = pred.permute(1,0,2)
        test_loss += loss_fn(pred, x[:,::35,:]).item()
    train_loss /= len(train_loader)
    test_loss /= len(test_loader)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    print(i, default_timer()-t, train_loss, test_loss)